In [1]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt

import sys
from pathlib import Path

# 1) Ensure the repo parent is on sys.path (..../<parent>/Maurice2)
repo_parent = Path.cwd().resolve().parents[0]
if str(repo_parent) not in sys.path:
    sys.path.insert(0, str(repo_parent))

# 2) (Optional) If you want to override the data dir at runtime, set this:
# os.environ["MAURICE2_DATA_DIR"] = str(Path(repo_parent, "Maurice2", "data"))

# 3) Autoreload to pick up edits to Maurice2/*
%load_ext autoreload
%autoreload 2

from dynamics import Dynamics